In [200]:
import os.path as path # used for easily finding the csvs in other directories
from PyPDF2 import PdfReader # used to read and extract text from PDFs

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from datetime import datetime
import re

In [201]:
# loading csv from local directory function
def load_within_directory(directory_string):
    """
    Takes in a string of where your csv is located in the repo folder and turns
    into a dataframe using pandas read_csv

    example directory string: '/data/raw_data/CIE/client_needs_table.csv'
    :param directory_string: string containing file with directory desired
    :return: a dataframe
    """

    temp_path = path.abspath(path.join("data" ,"../../..")) # finds the parent directory

    # concatenates with directory
    return temp_path + directory_string

# imported from library cleaning script
def clean_data(df):
    return (df
        .replace(r'^s*$', float('NaN'), regex = True)  # There were blank rows, so first I changed them to "NaN" values
        .dropna(axis=0, how='all')  # Then dropped any row where all the values were NaN
        .assign(street=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.rsplit('#', n=1, expand=True)[0].str.rsplit(',', expand=True)[0], # Get street data. Assumes # or comma or nothing
                unit1=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.extract(r'(\d+$)|(\#\w+$)')[1].str.strip('#'), # Get unit data
                unit2=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.rsplit('#', n=1, expand=True)[1], # Get unit data
                unit3=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.lower().str.rsplit('unit', n=1, expand=True)[1].str.strip().str.lstrip('#'), # Get unit data
                unit4=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.lower().str.rsplit('apt', n=1, expand=True)[1].str.strip().str.lstrip('#'), # Get unit data
                unit5=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.lower().str.rsplit('suite', n=1, expand=True)[1].str.strip().str.lstrip('#'), # Get unit data
                city=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.split(',', n=1, expand=True)[0],
                state=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.split(',', n=1, expand=True)[1].str.rsplit(' ', n=1, expand=True)[0].str.strip(),
                zip=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.split(',', n=1, expand=True)[1].str.rsplit(' ', n=1, expand=True)[1].str.strip())
    )

In [202]:
# loading sheriff pdf
sheriff_pdf = PdfReader(load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/sdso_lockout_service_activity_details_jan_2018_jan_2023.pdf'))

In [203]:
# this cell is just to detail what will go on in the pdf
# parsing loop

# We need to check the length of pages in the file
print(len(sheriff_pdf.pages)) # should be 1184

# grab a specific page from the pdf file
temp_page = sheriff_pdf.pages[0]

# then extract the text from page and save it
temp_text = temp_page.extract_text()
print(temp_text)

1184
SAN DIEGO COUNTY SHERIFF'S OFFICE
Eviction List
TO 01/01/2018 01/31/2023
Office Phone #: (619) 544-6401
Address Status File Number Occupants Restoration Date Time
Chula Vista
2015250864 NOTFOUND Alex Luevano,All Unknown 
Occupants,Carlos 
Luevano1256 8th Street
(Front and Back House)
Imperial Beach, CA 9193204/21/2021  12:00 AM
2015250883 CANCELLED Audra Souza, Anthony A . 
Souza464 E H St #503
Chula Vista , CA 9191005/20/2020  12:00 AM
2017253065 SERVED Maria Varela ,Luis Matheu 
III1754 Via Costina
San Diego, CA 9217301/04/2018  12:00 AM
2017253067 SERVED All Unknown 
Occupants, Charvella West1357 Burgundy Dr
Chula Vista , CA 9191301/04/2018  12:00 AM
2017253088 SERVED Hildelisa Ochoa,All 
Unknown Occupants918 Tenth St , #3
Coronado, CA 9211801/03/2018  12:00 AM
2017253090 SERVED All Unknown 
Occupants,Raphael 
Vazquez ,Vanessa Rachel 
Vazquez1746 Via Capri
Chula Vista , CA 9191301/03/2018  12:00 AM
2017253147 SERVED Rogel Trucking , 
LLC,Jeremias Rogel Jr,All 
Unknown Occupants

# SDSO PDF Extraction Tool
## How this works
* First, the PDF is read from the above code, luckily it is fairly light on images so it is no larger than 4 MB
* Second, the loop below will scrape every single page of text and store it into a large object (like a big text file)
    * This makes it easier to parse through and format into a dataframe
* Third, we need to make a string parser loop which will store a dictionary of each "column" for the dataframe. This is a bit difficult as we will essentially need to figure out how many lines each "row" this takes up.
    * The parser divides the giant text document into rows, then adds them to each respective column pieceewise
* Each column will be merged into a single dataframe with the hopes that it will be much easier to analyse.
---

In [204]:
# making a string object to append to
raw_pdf_string = ""

for i in range(0, len(sheriff_pdf.pages)):
    temp_page = sheriff_pdf.pages[i]
    temp_text = temp_page.extract_text()
    raw_pdf_string = raw_pdf_string + temp_text


So the format for each row should be:

Chula Vista

2015250864 NOTFOUND Alex Luevano,All Unknown

Occupants,Carlos

Luevano1256 8th Street

(Front and Back House)

Imperial Beach, CA 9193204/21/2021  12:00 AM

* First we need to remove the initial title
* Then try chunking them out into a proper row or column header/unneeded info

---

In [205]:
# converting lines into a list to index through
raw_pdf_list = raw_pdf_string.splitlines()
del raw_pdf_list[0:6] # deleting the police header to make it easier to process
# very odd "Chula Vista" entry? Was this a mistake?

In [206]:
# okay, so we can see that the items starting with "Address" should be removed
# any lines beginning with "Printed:" also must be removed
# look at this for an example, 'Printed:' in raw_pdf_list[46]
for i in reversed(range(len(raw_pdf_list))): # reversing through list as dropping indeces causes issues
    if 'Printed:' in raw_pdf_list[i]:
        del raw_pdf_list[i]
# now we have no more headings or needless info other than rows!

In [207]:
# next, we need to split up all of these into row cells
# the best way would be to look for the file no. and end at the time
# raw_pdf_list[0][0:2] to check for 20 or 30
# raw_pdf_list[4][-2::1] # to check for AM or PM
raw_rows = []
temp_string = ""

for string in raw_pdf_list:
    temp_string = temp_string + string
    if (string[-2::1] == "AM") or (string[-2::1] == "PM"):
        raw_rows.append(temp_string)
        temp_string = ""

# also holy moly, is it just an error or laziness?
# all the recorded Times as 12:00AM...

In [208]:
# first lets get the case no.
# easy enough, just strip the first 10 digits from the left
case_no_list = []
for i in range(0, len(raw_rows), 1):
    case_no = raw_rows[i][0:10]
    case_no_list.append(case_no)
    raw_rows[i] = raw_rows[i].replace(case_no, '')
    raw_rows[i] = raw_rows[i].lstrip()

In [209]:
# next for the case statuses
status_list = []
for i in range(0, len(raw_rows), 1):
    all_words = raw_rows[i].split()
    status = all_words[0]
    del all_words[0]
    raw_rows[i] = ' '.join(all_words)
    status_list.append(status)

In [210]:
# next are the dates
# raw_rows[0][-19::1] is the format
date_list = []
for i in range(0, len(raw_rows), 1):
    date = raw_rows[i][-19::1]
    date_list.append(date)
    raw_rows[i] = raw_rows[i].replace(date, '')
    raw_rows[i] = raw_rows[i].rstrip()

In [211]:
# current progress, almost done, just need to split the
# names and addresses. This Might be a little tricky...

# For now, here is the current dataframe, we will instead add in the columns
# cleaned from the library's method while also doing other forms of cleaning
dataframe_dict = {"File_Number":case_no_list, "Raw_Rows":raw_rows, "Restoration_Date":date_list, "Status":status_list}
sheriff_df = pd.DataFrame(dataframe_dict)

# loading library df from EvictionPDF_altered script
library_df = pd.read_csv(load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/evictions_library_export.csv'))


# there are noticeably less rows than the other method
# this could be from duplicates, so let's account for those

# per each column
sheriff_df = sheriff_df.drop_duplicates(subset = ['File_Number'], keep = 'first', ignore_index=True)

# oddly less library, so let's account for them in that df too
library_df = library_df.drop_duplicates(subset = ['File_Number'], keep = 'first', ignore_index=True)

# let's check
print("len sheriff_df", len(sheriff_df), "len library_df",len(library_df))

len sheriff_df 12799 len library_df 13926


In [212]:
# alright, so while the original best course of action would be to merge
# pdf plumber's extraction uses a very threshold-based extraction where
# we do not have exact benchmarks to judge how well it is identifying the table contents
# in an effort to be as precise as possible, we will have to go with sheriff_df and use a string
# cleaning method instead (using regex)

# making an empty column to add values
sheriff_df["Address"] = ""

# loop through all raw rows with names and addresses. Almost all but 1 addresses
for i in range(0, len(sheriff_df)):
    temp_string = sheriff_df["Raw_Rows"][i] # pulling the string
    numeric_found = re.search(r"\d", temp_string) # locating the first instance of a numeric
    if numeric_found: # if we found the address
        address_string = temp_string[numeric_found.start():len(temp_string)] # slice it out
        sheriff_df["Address"][i] = address_string # add it to the row under the new column
        sheriff_df["Raw_Rows"][i] = sheriff_df["Raw_Rows"][i].replace(address_string, '') # and remove it from the raw_row
    else: # if not
        sheriff_df["Address"][i] = "Warning! Address Now Found!" # add a warning, do not mess with the raw data

In [213]:
# couple things left, splitting Time off from the date so that we can feed
# this df into the library function

sheriff_df["Time"] = ""

for i in range(0, len(sheriff_df)):
    time = sheriff_df["Restoration_Date"][i][-8::1]
    sheriff_df["Time"][i] = time
    sheriff_df["Restoration_Date"][i] = sheriff_df["Restoration_Date"][i].replace(time, '')
    sheriff_df["Restoration_Date"][i] = sheriff_df["Restoration_Date"][i].rstrip()

In [214]:
# to use the library script, we need our columns to match this order
# File_Number	Occupants	Address	Restoration_Date	Time	Status
# so re_ordering and renaming the columns
sheriff_df = sheriff_df.rename({'Raw_Rows':'Occupants'}, axis='columns') # renaming raw_rows to Occupants

# reordering
sheriff_df = sheriff_df[['File_Number', 'Occupants', 'Address', 'Restoration_Date', 'Time', 'Status']]

In [215]:
# cleaning using the library script
# NOTE: due to some of the digits not being fully seperable from the aaddress, this means
# the script is unable to fully divide, so for now it will just make mistakes
# this can be fixed, will just take some more time.

sheriff_df_library_cleaned = clean_data(sheriff_df)

In [216]:
# exporting all dataframes

# let's do the library_hybrid first (not recommended for use)
directory = load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/')
directory = directory + 'cleaned_evictions_library_hybrid_export.csv'
sheriff_df_library_cleaned.to_csv(directory, index=False)

# next our own version (better cleaned)
directory = load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/')
directory = directory + 'cleaned_evictions_export.csv'
sheriff_df.to_csv(directory, index=False)

# and finally the library df with removed duplicates
directory = load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/')
directory = directory + 'evictions_library_no_duplicates_export.csv'
library_df.to_csv(directory, index=False)

In [217]:
# sheriff_df.loc[sheriff_df['File_Number'] == "2019453282"]